<a href="https://colab.research.google.com/github/kdemertzis/EKPA/blob/main/ChatGPT_Incident_Response.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#εγκατάσταση πακέτου transformers (https://pypi.org/project/transformers/)
!pip install transformers
#εγκατάσταση πακέτου requests (https://pypi.org/project/requests/)
!pip install requests


In [10]:
#εισαγωγή βιβλιοθηκών - πακέτων
import pandas as pd
import torch
import random
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from transformers import GPT2LMHeadModel, GPT2Tokenizer

#aπλό μοντέλο IDS για σκοπούς επίδειξης
class SimpleIDS:
    def __init__(self):
        #aρχικοποιήση ενός φορέα TF-IDF και ενός ταξινομητή Naive Bayes
        self.vectorizer = TfidfVectorizer()
        self.classifier = MultinomialNB()

    def train(self, data, labels):
        #εκπαίδευση μοντέλου
        vectorized_data = self.vectorizer.fit_transform(data)
        self.classifier.fit(vectorized_data, labels)

    def predict(self, text):
        #πρόβλεψη για τον εάν υπάρχει ή όχι επίθεση
        vectorized_text = self.vectorizer.transform([text])
        return self.classifier.predict(vectorized_text)[0]

#ενσωμάτωση ChatGPT για την υλοποίηση απόκρισης συμβάντων
class EnhancedChatGPTIncidentResponse:
    def __init__(self):
        #φόρτωση προεκπαιδευμένου μοντέλου ChatGPT
        self.tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
        self.model = GPT2LMHeadModel.from_pretrained("gpt2")

    def generate_response(self, incident_identification, incident_summary, incident_notification, actions):
        #δημιουργία μιας αναγνώσιμης από άνθρωπο απάντησης χρησιμοποιώντας το ChatGPT με ρητές ρυθμίσεις
        prompt = (
            f"INCIDENT IDENTIFICATION INFORMATION: {incident_identification}\n"
            f"INCIDENT SUMMARY: {incident_summary}\n"
            f"INCIDENT NOTIFICATION – OTHERS: {incident_notification}\n"
            f"ACTIONS: {actions}\n"
        )

        #προσαρμογή της προτροπής
        input_ids = self.tokenizer.encode(prompt, return_tensors="pt", max_length=150, truncation=True)

        #ρύθμιση της μάσκας προσοχής
        attention_mask = torch.ones(input_ids.shape, device=input_ids.device)

        #δημιουργία απάντησης με ρητές ρυθμίσεις
        output = self.model.generate(
            input_ids,
            attention_mask=attention_mask,
            max_length=300,
            num_return_sequences=1,
            no_repeat_ngram_size=2,
            pad_token_id=self.tokenizer.eos_token_id
        )

        #αποκοδικοποίηση της εξόδου
        response = self.tokenizer.decode(output[0], skip_special_tokens=True)
        return response

#δημιουργία συνθετικών δεδομένων
def generate_synthetic_data(num_samples):
    incident_identifications = [f"ID{i}" for i in range(1, num_samples + 1)]
    incident_summaries = ["Normal activity"] * (num_samples // 2) + ["SQL injection attempt"] * (num_samples // 2)
    incident_notifications = ["Security Team"] * num_samples
    actions = ["Monitor"] * (num_samples // 2) + ["Isolate and Investigate"] * (num_samples // 2)

    synthetic_data = pd.DataFrame({
        'IncidentIdentification': incident_identifications,
        'IncidentSummary': incident_summaries,
        'IncidentNotification': incident_notifications,
        'Actions': actions
    })

    return synthetic_data

#παράδειγμα χρήσης
if __name__ == "__main__":
    ids_model = SimpleIDS()
    enhanced_chatgpt_response = EnhancedChatGPTIncidentResponse()

    #παραγωγή δεδομένων σε ροές (batches)
    num_samples = 10
    synthetic_data = generate_synthetic_data(num_samples)

    #εκπαίδευση του IDS
    training_data = ["Normal network traffic", "SQL injection attempt", "Malicious file download"]
    training_labels = ["Normal", "Attack", "Attack"]

    try:
        ids_model.train(training_data, training_labels)
    except Exception as e:
        print(f"Error during training: {e}")

    #τύποι επιθέσεων
    attack_types = [
        "SQL injection attempt",
        "Malicious file download",
        "Ransomware",
        "DDoS attack",
        "Cryptojacking",
        "Phishing Attack",
        "Man-in-the-Middle Attack"
    ]

    #προσομοίωση εισερχόμενης κίνησης δικτύου και δημιουργία αποκρίσεων συμβάντων
    for index, row in synthetic_data.iterrows():
        incident_identification = row['IncidentIdentification']
        incident_summary = random.choice(attack_types)
        incident_notification = row['IncidentNotification']
        actions = row['Actions']

        #χρήση του IDS για την πρόβλεψη επιθέσεων
        if ids_model.predict(incident_summary) == "Attack":
            #εφόσον εντοπισθεί επίθεση δημιουργία απόκρισης συμβάντος
            print("++++++++++ Incident Report ++++++++++")
            print(synthetic_data.loc[[index]])
            print("++++++++++ Incident Response ++++++++++")
            response = enhanced_chatgpt_response.generate_response(
                incident_identification, incident_summary, incident_notification, actions
            )
            print(response)
            print("+" * 40)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


++++++++++ Incident Report ++++++++++
  IncidentIdentification  IncidentSummary IncidentNotification  Actions
0                    ID1  Normal activity        Security Team  Monitor
++++++++++ Incident Response ++++++++++
INCIDENT IDENTIFICATION INFORMATION: ID1
INCIDENT SUMMARY: Malicious file download
INCIDENT NOTIFICATION – OTHERS: Security Team
ACTIONS: Monitor
MULTIPLE MULTI-SERVICE MACHINE
SUMMERS

CUSTOMER SERVICE
.NET
:
-
AUTHOR: John D. D'Amato
DATE: May 1, 2013
METHOD OF ACTION: File download.
DESCRIPTION: This is a malicious file that downloads files from a remote server. The file is located in the /var/www/html/ directory. It is not a legitimate file. This file does not contain any information about the file, but it is an attempt to download files that are not in a directory that is accessible by the user. If the malicious files are found, the files will be deleted. A malicious user can download the downloaded files by clicking on the download button. Once the downloads are